In [3]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load the dataset
file_path = 'heart.csv'
data = pd.read_csv(file_path)

# Parameters for ACO
num_ants = 10               # Number of ants in each iteration
num_features = data.shape[1] - 1  # Number of features (excluding target)
num_iterations = 20         # Number of iterations
evaporation_rate = 0.2      # Evaporation rate for pheromone update
alpha = 1.0                 # Influence of pheromone on feature selection
beta = 2.0                  # Influence of heuristic (accuracy) on feature selection
initial_pheromone = 1.0     # Initial pheromone level

# Initial pheromone levels for each feature
pheromone_levels = np.full(num_features, initial_pheromone)

# Split data into features and target
X = data.drop('target', axis=1)
y = data['target']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Initialize best accuracy and feature subset
best_accuracy = 0.0
best_features = None

# ACO Algorithm
for iteration in range(num_iterations):
    all_accuracies = []     # To store accuracy of each ant
    ant_solutions = []      # To store selected features for each ant

    # Each ant selects a subset of features
    for ant in range(num_ants):
        selected_features = np.random.rand(num_features) < pheromone_levels / pheromone_levels.sum()
        if not any(selected_features):
            selected_features[np.random.randint(0, num_features)] = True

        # Train and evaluate the classifier on the selected features
        selected_X_train = X_train.loc[:, selected_features]
        selected_X_test = X_test.loc[:, selected_features]

        knn = KNeighborsClassifier(n_neighbors=3)
        knn.fit(selected_X_train, y_train)
        y_pred = knn.predict(selected_X_test)
        accuracy = accuracy_score(y_test, y_pred)

        # Update if this ant's solution is the best so far
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_features = selected_features

        all_accuracies.append(accuracy)
        ant_solutions.append(selected_features)

    # Update pheromones based on ant solutions
    for i in range(num_features):
        pheromone_levels[i] *= (1 - evaporation_rate)  # Evaporation

        # Reinforce pheromone for features in better solutions
        for ant in range(num_ants):
            if ant_solutions[ant][i]:
                pheromone_levels[i] += all_accuracies[ant] / best_accuracy

print("Best accuracy achieved:", best_accuracy)
print("Best feature subset (1 = selected, 0 = not selected):", best_features.astype(int))

Best accuracy achieved: 0.9025974025974026
Best feature subset (1 = selected, 0 = not selected): [0 0 0 0 1 1 0 0 0 0 0 1 1]
